In [993]:
import pandas as pd
import chardet
import os
import re
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from pandas import Period
import numpy as np

pd.set_option("display.max_columns", None)

### Tool

In [994]:
all_columns = ["ASIN", "Nom du produit", "Marque", "COGS", "COGS evol LM", "COGS evol SPLY", "Unités", "Unités evol LM", "Unités evol SPLY", "Retours", "Retours evol LM", "Retours evol SPLY", "COGS LM", "Unités LM", "Retours LM", "is_in_vendor_central", "Item no.", "EAN", "Item", "Extras", "Size", "Description Item", "Cat. 1", "Cat. 2", "Cat. 3", "Cat. 4", "Visible", "Title Content", "Variations", "Brand store URL", "Images", "Videos", "AI summary", "Code", "Catalogue", "Ads Impressions CM", "Ads Clicks CM", "Ads CTR CM", "Ads Units 14d CM", "Ads Costs CM", "Ads RoAS CM", "Ads CVR CM", "Sell-out CM", "Sell-out PM", "Total sell-out CM", "Total sell-out PM", "Revenue CM", "Revenue PM", "Units CM", "Units PM", "SRP", "Margin", "Replacements CM", "Replacements PM", "Stock", "Stock value", "Total stock", "Total stock value", "Reviews", "Stars", "Coverage", "Rank 1", "Buy Box", "Buy Box PM", "Days not Buy Box", "Views CM", "Views PM", "CVR CM", "CVR PM", "is_in_amvisor", "extract_date"]

In [995]:
restricted = ['ASIN', "Nom du produit", "EAN", "COGS", 'Unités', 'Retours', 'is_in_vendor_central', 'is_in_amvisor', 'Cat. 1', 'Cat. 2', 'Cat. 3', 'Cat. 4']

In [996]:
interesting_columns = ["ASIN", "Nom du produit", "COGS", "COGS SPLY (€)", "COGS diff SPLY (€)", "COGS SPLY (%)", "Unités", "SRP"]

### Load data

In [997]:
df = pd.read_csv("../data/all_data.csv", dtype={"AI summary": str, "EAN": str}, low_memory=False)
df = df.sort_values(["extract_date", "COGS"], ascending=False)
print(df.shape)
df.head(3)

(34147, 77)


,ASIN,Nom du produit,Marque,COGS,COGS LM (%),COGS SPLY (%),Unités,Unités LM (%),Unités SPLY (%),Retours,Retours LM (%),Retours SPLY (%),Source (Fabrication ou Approvisionnement),is_in_vendor_central,Item no.,EAN,Item,Extras,Size,Description Item,Cat. 1,Cat. 2,Cat. 3,Cat. 4,Visible,Title Content,Variations,Brand store URL,Images,Videos,AI summary,Code,Catalogue,Ads Impressions CM,Ads Clicks CM,Ads CTR CM,Ads Units 14d CM,Ads Costs CM,Ads RoAS CM,Ads CVR CM,Sell-out CM,Sell-out PM,Total sell-out CM,Total sell-out PM,Units CM,Units PM,SRP,Margin,Replacements CM,Replacements PM,Stock,Stock value,Total stock,Total stock value,Reviews,Avg. stars,Stars,Coverage,Rank 1,Rank 2,Buy Box,Buy Box PM,Days not Buy Box,Views CM,Views PM,CVR CM,CVR PM,is_in_amvisor,Net PPM,Net PPM SPLY,extract_date,annee,mois,COGS SPLY (€),COGS diff SPLY (€),Category 1,Category 2
23,B00005QFCJ,Hama Adaptateur Cassette VHS-C/VHS (Convertiss...,Hama,10142.55,-7.30,63.37,495.0,-7.30,63.37,64.0,-31.91,45.45,1_fab,True,00044704,4007249447043,Hama Cassette adaptatrice VHS-C/VHS motorisée,NaN,NaN,REGULAR,Photo & Accessories,Studio & Diverse,Camera,Adapters,Yes,Hama Adaptateur Cassette VHS-C/VHS (Convertiss...,B00005QFCJ,NaN,13.0,2.0,Les clients apprécient le bon rapport qualité-...,PR,STO,66234.0,1550.0,2.3402,225.0,200.90,27.36,14.52,0.00,0.00,9302.46,10941.66,0.0,0.0,41.99,19.8,0.0,0.0,0.0,0.0,293.0,6003.57,901.0,4.0,3926.0,NaN,1304.0,221.0,Amazon,100.0,NaN,8037.0,8960.0,5.65,5.96,True,39.35,42.289092,2025-03,2025,3,6208.47,3934.08,Photo & Accessories,Studio & Diverse
82,B00006JAQ7,"Hama 007103 - Ruban photo transparent, Blanc, ...",Hama,3787.44,105.46,851.83,1313.0,102.94,1041.74,11.0,1000.00,NaN,1_fab,True,00007103,4007249071033,Hama Distributeur de bandes adhésives,NaN,2 x 500 bandes,REGULAR,Picture World,Storage & Design,Photo Storage & Presentation,Storage & Archiving,Yes,"Hama 007103 - Ruban photo transparent, Blanc, ...",B00005YVIU B00006JAQ7 B0CPF2BMJL,NaN,5.0,2.0,Les clients apprécient la super adhérence des ...,PR,STO,90395.0,1144.0,1.2656,552.0,526.34,4.13,48.25,884.83,394.44,3509.31,1843.40,305.0,136.0,3.99,68.1,0.0,0.0,356.0,1032.4,786.0,2278.34,2000.0,4.6,12826.0,1.1,398.0,1.0,Amazon,58.8,NaN,2642.0,1730.0,46.06,37.40,True,35.85,52.236631,2025-03,2025,3,397.91,3389.53,Picture World,Storage & Design
32,B00005YVIU,"Hama Distributeurs Coins photo (1000 coins, au...",Hama,3035.01,114.29,7188.69,1044.0,113.93,8600.00,9.0,NaN,NaN,1_fab,True,00007108,4007249071088,Hama Boîte distributrice coins adhésifs pr photos,Transparent,2x500 soins,REGULAR,Picture World,Storage & Design,Photo Storage & Presentation,Storage & Archiving,Yes,"Hama Distributeurs Coins photo (1000 coins, au...",B00005YVIU B00006JAQ7 B0CPF2BMJL,NaN,7.0,1.0,Les clients apprécient la super adhérence des ...,PR,PUA,184208.0,1878.0,1.0195,738.0,1144.90,2.76,39.30,1058.50,124.70,2834.91,1416.30,365.0,43.0,3.99,70.2,0.0,0.0,167.0,484.3,552.0,1600.80,2002.0,4.6,12827.0,0.4,273.0,1.0,Amazon,58.8,NaN,2494.0,1249.0,39.09,39.07,True,38.82,54.560787,2025-03,2025,3,41.64,2993.37,Picture World,Storage & Design


In [998]:
stocks = pd.read_csv("../data/stock_units.csv", low_memory=False)
print(stocks.shape)
stocks.head(3)

(1485, 456)


,2024-01-01,2024-01-02,2024-01-03,2024-01-04,2024-01-05,2024-01-06,2024-01-07,2024-01-08,2024-01-09,2024-01-10,2024-01-11,2024-01-12,2024-01-13,2024-01-14,2024-01-15,2024-01-16,2024-01-17,2024-01-18,2024-01-19,2024-01-20,2024-01-21,2024-01-22,2024-01-23,2024-01-24,2024-01-25,2024-01-26,2024-01-27,2024-01-28,2024-01-29,2024-01-30,2024-01-31,2024-02-01,2024-02-02,2024-02-03,2024-02-04,2024-02-05,2024-02-06,2024-02-07,2024-02-08,2024-02-09,2024-02-10,2024-02-11,2024-02-12,2024-02-13,2024-02-14,2024-02-15,2024-02-16,2024-02-17,2024-02-18,2024-02-19,2024-02-20,2024-02-21,2024-02-22,2024-02-23,2024-02-24,2024-02-25,2024-02-26,2024-02-27,2024-02-28,2024-02-29,2024-03-01,2024-03-02,2024-03-03,2024-03-04,2024-03-05,2024-03-06,2024-03-07,2024-03-08,2024-03-09,2024-03-10,2024-03-11,2024-03-12,2024-03-13,2024-03-14,2024-03-15,2024-03-16,2024-03-17,2024-03-18,2024-03-19,2024-03-20,2024-03-21,2024-03-22,2024-03-23,2024-03-24,2024-03-25,2024-03-26,2024-03-27,2024-03-28,2024-03-29,2024-03-30,2024-03-31,2024-04-01,2024-04-02,2024-04-03,2024-04-04,2024-04-05,2024-04-06,2024-04-07,2024-04-08,2024-04-09,2024-04-10,2024-04-11,2024-04-12,2024-04-13,2024-04-14,2024-04-15,2024-04-16,2024-04-17,2024-04-18,2024-04-19,2024-04-20,2024-04-21,2024-04-22,2024-04-23,2024-04-24,2024-04-25,2024-04-26,2024-04-27,2024-04-28,2024-04-29,2024-04-30,2024-05-01,2024-05-02,2024-05-03,2024-05-04,2024-05-05,2024-05-06,2024-05-07,2024-05-08,2024-05-09,2024-05-10,2024-05-11,2024-05-12,2024-05-13,2024-05-14,2024-05-15,2024-05-16,2024-05-17,2024-05-18,2024-05-19,2024-05-20,2024-05-21,2024-05-22,2024-05-23,2024-05-24,2024-05-25,2024-05-26,2024-05-27,2024-05-28,2024-05-29,2024-05-30,2024-05-31,2024-06-01,2024-06-02,2024-06-03,2024-06-04,2024-06-05,2024-06-06,2024-06-07,2024-06-08,2024-06-09,2024-06-10,2024-06-11,2024-06-12,2024-06-13,2024-06-14,2024-06-15,2024-06-16,2024-06-17,2024-06-18,2024-06-19,2024-06-20,2024-06-21,2024-06-22,2024-06-23,2024-06-24,2024-06-25,2024-06-26,2024-06-27,2024-06-28,2024-06-29,2024-06-30,2024-07-01,2024-07-02,2024-07-03,2024-07-04,2024-07-05,2024-07-06,2024-07-07,2024-07-08,2024-07-09,2024-07-10,2024-07-11,2024-07-12,2024-07-13,2024-07-14,2024-07-15,2024-07-16,2024-07-17,2024-07-18,2024-07-19,2024-07-20,2024-07-21,2024-07-22,2024-07-23,2024-07-24,2024-07-25,2024-07-26,2024-07-27,2024-07-28,2024-07-29,2024-07-30,2024-07-31,2024-08-01,2024-08-02,2024-08-03,2024-08-04,2024-08-05,2024-08-06,2024-08-07,2024-08-08,2024-08-09,2024-08-10,2024-08-11,2024-08-12,2024-08-13,2024-08-14,2024-08-15,2024-08-16,2024-08-17,2024-08-18,2024-08-19,2024-08-20,2024-08-21,2024-08-22,2024-08-23,2024-08-24,2024-08-25,2024-08-26,2024-08-27,2024-08-28,2024-08-29,2024-08-30,2024-08-31,2024-09-01,2024-09-02,2024-09-03,2024-09-04,2024-09-05,2024-09-06,2024-09-07,2024-09-08,2024-09-09,2024-09-10,2024-09-11,2024-09-12,2024-09-13,2024-09-14,2024-09-15,2024-09-16,2024-09-17,2024-09-18,2024-09-19,2024-09-20,2024-09-21,2024-09-22,2024-09-23,2024-09-24,2024-09-25,2024-09-26,2024-09-27,2024-09-28,2024-09-29,2024-09-30,2024-10-01,2024-10-02,2024-10-03,2024-10-04,2024-10-05,2024-10-06,2024-10-07,2024-10-08,2024-10-09,2024-10-10,2024-10-11,2024-10-12,2024-10-13,2024-10-14,2024-10-15,2024-10-16,2024-10-17,2024-10-18,2024-10-19,2024-10-20,2024-10-21,2024-10-22,2024-10-23,2024-10-24,2024-10-25,2024-10-26,2024-10-27,2024-10-28,2024-10-29,2024-10-30,2024-10-31,2024-11-01,2024-11-02,2024-11-03,2024-11-04,2024-11-05,2024-11-06,2024-11-07,2024-11-08,2024-11-09,2024-11-10,2024-11-11,2024-11-12,2024-11-13,2024-11-14,2024-11-15,2024-11-16,2024-11-17,2024-11-18,2024-11-19,2024-11-20,2024-11-21,2024-11-22,2024-11-23,2024-11-24,2024-11-25,2024-11-26,2024-11-27,2024-11-28,2024-11-29,2024-11-30,2024-12-01,2024-12-02,2024-12-03,2024-12-04,2024-12-05,2024-12-06,2024-12-07,2024-12-08,2024-12-09,2024-12-10,2024-12-11,2024-12-12,2024-12-13,2024-12-14,2024-12-15,2024-12-16,2024-12-17,2024-12-18,2024-12-19,2024-12-20,2024-12-21,2024-12-22,2024-12-23,2024-12-24,2024-12-25,2024-12-26,2024-12-27,2024-12-28,2024-1

### Exploration

##### Category analysis Sell-out

In [999]:
dt = df

dt['extract_date'] = pd.to_datetime(dt['extract_date'], format='%Y-%m')
dt['year'] = dt['extract_date'].dt.year
dt['quarter'] = dt['extract_date'].dt.quarter
dt['year_quarter'] = df['year'].astype(str) + '-Q' + dt['quarter'].astype(str)

dt = dt.groupby(["year_quarter", "Category 1"], dropna=False).agg({"COGS": "sum", "Unités": "sum"}).reset_index().sort_values(["year_quarter", "COGS"], ascending=[False, False]).reset_index()

dt_Q12025 = dt[dt["year_quarter"] == "2025-Q1"]
dt_Q12024 = dt[dt["year_quarter"] == "2024-Q1"]

# On renomme d'abord les colonnes pour chaque dataframe
dt_Q12024 = dt_Q12024.rename(columns={
    "Category 1": "Cat. 1",
    "Unités": "QTY YTQ1 '24",
    "COGS": "COGS YTQ1 '24"
})

dt_Q12025 = dt_Q12025.rename(columns={
    "Category 1": "Cat. 1",
    "Unités": "QTY YTQ1 '25",
    "COGS": "COGS YTQ1 '25"
})

dt_Q12024['Cat. 1'] = dt_Q12024['Cat. 1'].fillna("Unknown")
dt_Q12025['Cat. 1'] = dt_Q12025['Cat. 1'].fillna("Unknown")

# Merger les deux sur Cat. 1
merged = pd.merge(dt_Q12024[['Cat. 1', "QTY YTQ1 '24", "COGS YTQ1 '24"]],
                  dt_Q12025[['Cat. 1', "QTY YTQ1 '25", "COGS YTQ1 '25"]],
                  on='Cat. 1',
                  how='outer')

# Calcul du % de changement sur les quantités
merged["% Change QTY"] = ((merged["QTY YTQ1 '25"] - merged["QTY YTQ1 '24"]) / merged["QTY YTQ1 '24"].replace(0, np.nan)) * 100
merged["% Change COGS"] = ((merged["COGS YTQ1 '25"] - merged["COGS YTQ1 '24"]) / merged["COGS YTQ1 '24"].replace(0, np.nan)) * 100
merged["€ Change QTY"] = ((merged["QTY YTQ1 '25"] - merged["QTY YTQ1 '24"])).replace(0, np.nan)
merged["€ Change COGS"] = ((merged["COGS YTQ1 '25"] - merged["COGS YTQ1 '24"])).replace(0, np.nan)
merged = merged.round(0)

# Optionnel : trier par variation ou autre critère
merged = merged.sort_values(by="% Change QTY", ascending=True)

merged = merged[["Cat. 1", "QTY YTQ1 '24", "QTY YTQ1 '25", "% Change QTY", "€ Change QTY", "COGS YTQ1 '24", "COGS YTQ1 '25", "% Change COGS", "€ Change COGS"]]
# Affichage du tableau
merged.reset_index(drop=True, inplace=True)
merged.sort_values("COGS YTQ1 '25", ascending=False)

,Cat. 1,QTY YTQ1 '24,QTY YTQ1 '25,% Change QTY,€ Change QTY,COGS YTQ1 '24,COGS YTQ1 '25,% Change COGS,€ Change COGS
6,Consumer Electronics,13770.0,9056.0,-34.0,-4714.0,124167.0,75802.0,-39.0,-48365.0
5,Picture World,23277.0,14255.0,-39.0,-9022.0,122255.0,70214.0,-43.0,-52041.0
8,Photo & Accessories,3183.0,2933.0,-8.0,-250.0,40396.0,45404.0,12.0,5008.0
4,Home & Office,6761.0,3692.0,-45.0,-3069.0,62887.0,34752.0,-45.0,-28135.0
2,Audio,1878.0,802.0,-57.0,-1076.0,45464.0,19007.0,-58.0,-26457.0
0,Computing & Tablet,3137.0,1112.0,-65.0,-2025.0,22186.0,9459.0,-57.0,-12726.0
1,Cables,4152.0,1528.0,-63.0,-2624.0,12734.0,6517.0,-49.0,-6217.0
10,Mobilcom,153.0,246.0,61.0,93.0,1973.0,3223.0,63.0,1250.0
7,Storage Media,490.0,370.0,-24.0,-120.0,3377.0,2714.0,-20.0,-663.0
3,Unknown,395.0,214.0,-46.0,-181.0,2582.0,2164.0,-16.0,-418.0


##### Top products

In [1000]:
dt = df

dt['extract_date'] = pd.to_datetime(dt['extract_date'], format='%Y-%m')
dt['year'] = dt['extract_date'].dt.year
dt['quarter'] = dt['extract_date'].dt.quarter
dt['year_quarter'] = df['year'].astype(str) + '-Q' + dt['quarter'].astype(str)

dt = dt.groupby(["year_quarter", "ASIN"], dropna=False).agg({"COGS": "sum", "Unités": "sum"}).reset_index().sort_values(["year_quarter", "COGS"], ascending=[False, False]).reset_index()

dt_Q12025 = dt[dt["year_quarter"] == "2025-Q1"]
dt_Q12024 = dt[dt["year_quarter"] == "2024-Q1"]

# On renomme d'abord les colonnes pour chaque dataframe
dt_Q12024 = dt_Q12024.rename(columns={
    "Product": "ASIN",
    "Unités": "QTY YTQ1 '24",
    "COGS": "COGS YTQ1 '24"
})

dt_Q12025 = dt_Q12025.rename(columns={
    "Product": "ASIN",
    "Unités": "QTY YTQ1 '25",
    "COGS": "COGS YTQ1 '25"
})

dt_Q12025.drop(["index"], inplace=True, axis=1)
dt_Q12024.drop(["index"], inplace=True, axis=1)

merged = pd.merge(dt_Q12024[['ASIN', "QTY YTQ1 '24", "COGS YTQ1 '24"]],
                  dt_Q12025[['ASIN', "QTY YTQ1 '25", "COGS YTQ1 '25"]],
                  on='ASIN',
                  how='outer')

# Calcul du % de changement sur les quantités
merged["% Change QTY"] = ((merged["QTY YTQ1 '25"] - merged["QTY YTQ1 '24"]) / merged["QTY YTQ1 '24"].replace(0, np.nan)) * 100
merged["% Change COGS"] = ((merged["COGS YTQ1 '25"] - merged["COGS YTQ1 '24"]) / merged["COGS YTQ1 '24"].replace(0, np.nan)) * 100
merged["€ Change QTY"] = ((merged["QTY YTQ1 '25"] - merged["QTY YTQ1 '24"])).replace(0, np.nan)
merged["€ Change COGS"] = ((merged["COGS YTQ1 '25"] - merged["COGS YTQ1 '24"])).replace(0, np.nan)
merged = merged.round(0)

print(f"Sort by Q1 2025 performance (COGS YTQ1 '25)")
merged = merged.sort_values("COGS YTQ1 '25", ascending=False).head(30)
top_products = merged.ASIN.unique()
merged

Sort by Q1 2025 performance (COGS YTQ1 '25)


,ASIN,QTY YTQ1 '24,COGS YTQ1 '24,QTY YTQ1 '25,COGS YTQ1 '25,% Change QTY,% Change COGS,€ Change QTY,€ Change COGS
0,B00005QFCJ,1057.0,21658.0,1605.0,32866.0,52.0,52.0,548.0,11208.0
27,B00D2S92L0,421.0,3019.0,974.0,6984.0,131.0,131.0,553.0,3965.0
24,B00006JAQ7,947.0,3165.0,2110.0,6065.0,123.0,92.0,1163.0,2900.0
12,B0BNLLF6YV,22.0,4993.0,29.0,5834.0,32.0,17.0,7.0,841.0
5,B0764LMPBN,737.0,7629.0,495.0,5128.0,-33.0,-33.0,-242.0,-2500.0
15,B00006JAXW,287.0,4555.0,397.0,4712.0,38.0,3.0,110.0,157.0
1029,B00006IVA8,1.0,11.0,515.0,4686.0,51400.0,41337.0,514.0,4675.0
465,B00005YVIU,34.0,118.0,1601.0,4654.0,4609.0,3845.0,1567.0,4536.0
2,B01N6NQ73J,1097.0,11698.0,419.0,4604.0,-62.0,-61.0,-678.0,-7094.0
32,B08GS8FVB5,126.0,2607.0,222.0,4073.0,76.0,56.0,96.0,1466.0


##### Worst products

In [1001]:
dt = df

dt['extract_date'] = pd.to_datetime(dt['extract_date'], format='%Y-%m')
dt['year'] = dt['extract_date'].dt.year
dt['quarter'] = dt['extract_date'].dt.quarter
dt['year_quarter'] = df['year'].astype(str) + '-Q' + dt['quarter'].astype(str)

dt = dt.groupby(["year_quarter", "ASIN"], dropna=False).agg({"COGS": "sum", "Unités": "sum"}).reset_index().sort_values(["year_quarter", "COGS"], ascending=[False, False]).reset_index()

dt_Q12025 = dt[dt["year_quarter"] == "2025-Q1"]
dt_Q12024 = dt[dt["year_quarter"] == "2024-Q1"]

# On renomme d'abord les colonnes pour chaque dataframe
dt_Q12024 = dt_Q12024.rename(columns={
    "Product": "ASIN",
    "Unités": "QTY YTQ1 '24",
    "COGS": "COGS YTQ1 '24"
})

dt_Q12025 = dt_Q12025.rename(columns={
    "Product": "ASIN",
    "Unités": "QTY YTQ1 '25",
    "COGS": "COGS YTQ1 '25"
})

dt_Q12025.drop(["index"], inplace=True, axis=1)
dt_Q12024.drop(["index"], inplace=True, axis=1)

merged = pd.merge(dt_Q12024[['ASIN', "QTY YTQ1 '24", "COGS YTQ1 '24"]],
                  dt_Q12025[['ASIN', "QTY YTQ1 '25", "COGS YTQ1 '25"]],
                  on='ASIN',
                  how='outer')

# Calcul du % de changement sur les quantités
merged["% Change QTY"] = ((merged["QTY YTQ1 '25"] - merged["QTY YTQ1 '24"]) / merged["QTY YTQ1 '24"].replace(0, np.nan)) * 100
merged["% Change COGS"] = ((merged["COGS YTQ1 '25"] - merged["COGS YTQ1 '24"]) / merged["COGS YTQ1 '24"].replace(0, np.nan)) * 100
merged["€ Change QTY"] = ((merged["QTY YTQ1 '25"] - merged["QTY YTQ1 '24"])).replace(0, np.nan)
merged["€ Change COGS"] = ((merged["COGS YTQ1 '25"] - merged["COGS YTQ1 '24"])).replace(0, np.nan)
merged = merged.round(0)

print(f"Sort by Q1 2025 performance (COGS YTQ1 '25)")
merged = merged.sort_values("€ Change COGS", ascending=True).head(30)
worst_products = merged.ASIN.unique()
merged

Sort by Q1 2025 performance (COGS YTQ1 '25)


,ASIN,QTY YTQ1 '24,COGS YTQ1 '24,QTY YTQ1 '25,COGS YTQ1 '25,% Change QTY,% Change COGS,€ Change QTY,€ Change COGS
1,B08SDR49RS,39.0,11769.0,0.0,0.0,-100.0,-100.0,-39.0,-11769.0
3,B0000AI2SY,951.0,8797.0,0.0,0.0,-100.0,-100.0,-951.0,-8797.0
2,B01N6NQ73J,1097.0,11698.0,419.0,4604.0,-62.0,-61.0,-678.0,-7094.0
4,B09MQQM1ZJ,162.0,8432.0,28.0,1459.0,-83.0,-83.0,-134.0,-6972.0
7,B0BL1SS1WK,803.0,6326.0,24.0,181.0,-97.0,-97.0,-779.0,-6145.0
9,B0000AIRY7,1356.0,5655.0,41.0,171.0,-97.0,-97.0,-1315.0,-5484.0
11,B08YS1PZWL,2412.0,5475.0,277.0,499.0,-89.0,-91.0,-2135.0,-4977.0
8,B00SL6I2WI,700.0,6159.0,213.0,1726.0,-70.0,-72.0,-487.0,-4433.0
17,B0026MT4CK,749.0,4259.0,11.0,63.0,-99.0,-99.0,-738.0,-4196.0
14,B08GSRV3PB,144.0,4796.0,24.0,800.0,-83.0,-83.0,-120.0,-3997.0


##### Reasons

In [1002]:
top_products
worst_products

array(['B08SDR49RS', 'B0000AI2SY', 'B01N6NQ73J', 'B09MQQM1ZJ',
       'B0BL1SS1WK', 'B0000AIRY7', 'B08YS1PZWL', 'B00SL6I2WI',
       'B0026MT4CK', 'B08GSRV3PB', 'B00006J4EP', 'B003PFBLOQ',
       'B097HN8V5V', 'B0009V6Q30', 'B00006JDEX', 'B07RHGG3S7',
       'B08SFG3JH5', 'B0764LMPBN', 'B0069MD6SQ', 'B0083M13TY',
       'B0053DXXJO', 'B08BNYFVFS', 'B00006J4GV', 'B00005YVGK',
       'B000EORX5W', 'B001F536RS', 'B00UV84E4O', 'B071ZVN9LP',
       'B0069MD6LI', 'B00AJKQ4YC'], dtype=object)

In [1003]:
dt = df

# 1. Produits hors catalogue
hors_catalogue = dt[~dt['is_in_amvisor']]

# 2. Évolution du nombre de reviews
reviews_evolution = dt.groupby(['ASIN', 'extract_date'])['Reviews'].sum().unstack()

# 3. Évolution de la note moyenne (Stars)
stars_evolution = dt.groupby(['ASIN', 'extract_date'])['Avg. stars'].mean().unstack()

# 4. Évolution du nombre de retours
retours_evolution = dt.groupby(['ASIN', 'extract_date'])['Retours'].sum().unstack(fill_value=0)

# 5. Évolution du CVR
cvr_evolution = dt.groupby(['ASIN', 'extract_date'])['CVR CM'].mean().unstack()

# 6. Évolution des ventes
sellout_evolution = dt.groupby(['ASIN', 'extract_date'])['COGS'].sum().unstack(fill_value=0)

# 7. Évolution du status
status_evolution = dt.groupby(['ASIN', 'extract_date'])['Code'].first().unstack()

# 8. Ads
ads_imp_evolution = dt.groupby(['ASIN', 'extract_date'])['Ads Impressions CM'].sum().unstack(fill_value=0)

# 9. Evolution du margin
margin_evolution = dt.groupby(['ASIN', 'extract_date'])['Margin'].first().unstack()

# 10. Evolution du rank
rank_evolution = dt.groupby(['ASIN', 'extract_date'])['Rank 2'].first().unstack()

# 10. Evolution du stock_fr
stock_fr_evolution = dt.groupby(['ASIN', 'extract_date'])['Stock'].first().unstack()

# 11. Evolution du stock_overall
stock_tot_evolution = dt.groupby(['ASIN', 'extract_date'])['Total stock'].first().unstack()

In [1004]:
def quarter_months(q):
        year, qtr = q.split('-')
        months = {
            'Q1': ['01', '02', '03'],
            'Q2': ['04', '05', '06'],
            'Q3': ['07', '08', '09'],
            'Q4': ['10', '11', '12']
        }[qtr]
        return [f"{year}-{m}" for m in months]

In [ ]:
def show_evol(df, product):
    return df[df.index == product]

def analyse_progression_reviews(df_reviews, q1: str, q2: str, seuil_pct=0):
    """
    Compare le nombre de reviews à la fin de deux trimestres (ex: '2024-Q1' vs '2025-Q1').

    Parameters:
        df_reviews : DataFrame avec dates en colonnes (pd.Timestamp ou str)
        q1, q2 : str ('YYYY-Qn') - ex: '2024-Q1', '2025-Q1'
        seuil_pct : seuil de pourcentage pour flag "Progression > X%"

    Returns:
        DataFrame avec reviews en fin de quarter, delta, % de croissance, flag, label
    """
    import numpy as np
    import pandas as pd

    # Convertir les noms de colonnes en chaînes
    colnames = df_reviews.columns.astype(str)

    # Dernier mois de chaque quarter
    last_month_q1 = quarter_months(q1)[-1]
    last_month_q2 = quarter_months(q2)[-1]

    # Récupérer les colonnes correspondantes
    review_end_q1 = df_reviews[[col for col in df_reviews.columns if last_month_q1 in str(col)][0]]
    review_end_q2 = df_reviews[[col for col in df_reviews.columns if last_month_q2 in str(col)][0]]


    delta_reviews_nb = (review_end_q2.fillna(0) - review_end_q1.fillna(0))
    pct_change = (delta_reviews_nb / review_end_q1.fillna(0)) * 100

    result = pd.DataFrame({
        f'Reviews end of {q1}': review_end_q1,
        f'Reviews end of {q2}': review_end_q2,
        'Reviews Gained': delta_reviews_nb,
        '% Change Reviews': pct_change,
        f'Progression > {seuil_pct}%': pct_change > seuil_pct,
        'check_reviews': np.where(pct_change > seuil_pct, "Augmentation", "No reviews")
    })

    return result

def analyse_progression_stars(df_stars, q1: str, q2: str, seuil=0.05):
    """
    Compare les notes moyennes entre la fin de deux quarters (ex: '2024-Q1' vs '2025-Q1').

    Parameters:
        df_stars : DataFrame avec dates en colonnes
        q1, q2 : str, ex: '2024-Q1', '2025-Q1'
        seuil : float, variation significative à partir de ce seuil

    Returns:
        DataFrame avec valeurs de fin de quarter, delta, tendance et flags.
    """
    import numpy as np
    import pandas as pd

    colnames = df_stars.columns.astype(str)

    last_month_q1 = quarter_months(q1)[-1]
    last_month_q2 = quarter_months(q2)[-1]

    # Récupérer les colonnes correspondantes
    stars_q1 = df_stars[[col for col in df_stars.columns if last_month_q1 in str(col)][0]]
    stars_q2 = df_stars[[col for col in df_stars.columns if last_month_q2 in str(col)][0]]

    delta_star = stars_q2 - stars_q1

    def tendance(val):
        if pd.isna(val):
            return "Inconnu"
        elif val > seuil:
            return "Amélioration"
        elif val < -seuil:
            return "Dégradation"
        else:
            return "Stable"

    tendance_eval = delta_star.apply(tendance)

    result = pd.DataFrame({
        f'Stars end of {q1}': stars_q1,
        f'Stars end of {q2}': stars_q2,
        'Delta Stars': delta_star,
        'Tendance': tendance_eval,
        'Delta significatif': delta_star.abs() > seuil,
        'check_stars': tendance_eval
    })

    return result

def analyse_progression_cvr(df_cvr, seuil_chute=3, seuil_bas=5, seuil_volatilite=10):
    import numpy as np
    import pandas as pd

    def get_first_valid(row):
        idx = row.first_valid_index()
        return row[idx] if idx is not None else np.nan

    def get_last_valid(row):
        idx = row.last_valid_index()
        return row[idx] if idx is not None else np.nan

    first_val = df_cvr.apply(get_first_valid, axis=1)
    last_val = df_cvr.apply(get_last_valid, axis=1)

    delta = last_val - first_val

    moyenne = df_cvr.mean(axis=1, skipna=True)
    ecart_type = df_cvr.std(axis=1, skipna=True)

    tendance = delta.apply(
        lambda x: "Hausse" if x > 0 else ("Baisse" if x < 0 else "Stable")
    )

    result = pd.DataFrame({
        'First CVR': first_val,
        'Last CVR': last_val,
        'Delta CVR': delta,
        'Tendance': tendance,
        'CVR Moyen': moyenne,
        'CVR Écart-type': ecart_type,
        'Chute significative': delta < -seuil_chute,
        'CVR globalement bas': moyenne < seuil_bas,
        'CVR instable': ecart_type > seuil_volatilite
    })

    return result

def analyse_progression_retours(df_retours, q_past: str, q_current: str, seuil_pct=10):
    """
    Compare les retours moyens entre deux quarters (ex: '2024-Q1' vs '2025-Q1').

    Parameters:
        df_retours : pd.DataFrame avec dates en colonnes (format datetime ou str)
        q_past, q2 : str, par exemple '2024-Q1', '2025-Q1'
        seuil_pct : float, seuil pour considérer une variation significative

    Returns:
        pd.DataFrame avec moyennes, delta, tendance et flags
    """

    # Convertir les colonnes en str pour être sûr
    colnames = df_retours.columns.astype(str)

    cols_q_past = [col for col in colnames if any(month in col for month in quarter_months(q_past))]
    cols_q_current = [col for col in colnames if any(month in col for month in quarter_months(q_current))]

    # Moyennes
    avg_q_past = df_retours[cols_q_past].mean(axis=1)
    avg_q_current = df_retours[cols_q_current].mean(axis=1)

    # Delta & % de changement
    delta = df_retours[cols_q_current].sum(axis=1) - df_retours[cols_q_past].sum(axis=1)
    delta_avg = avg_q_current - avg_q_past
    pct_change = (delta_avg / avg_q_past.replace(0, np.nan)) * 100

    # Fonction tendance
    def tendance(val):
        if val > seuil_pct:
            return "Hausse"
        elif val < -seuil_pct:
            return "Baisse"
        else:
            return "Stable"

    tendance_label = pct_change.apply(tendance)

    result = pd.DataFrame({
        f'Retours {q_past} (moy)': avg_q_past,
        f'Retours {q_current} (moy)': avg_q_current,
        'Delta': delta,
        '% Changement': pct_change,
        'Tendance': tendance_label,
        'Hausse problématique': delta > 0,
        'check_returns': tendance_label
    })

    return result

In [1006]:
product = "B08BNYFVFS"

##### Review

In [1007]:
evol = show_evol(reviews_evolution, product)
evol

extract_date,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01
ASIN,,,,,,,,,,,,,,,
B08BNYFVFS,0.0,0.0,0.0,41.0,42.0,42.0,42.0,45.0,0.0,101.0,101.0,101.0,101.0,101.0,101.0


In [1008]:
evol_analysis = analyse_progression_reviews(reviews_evolution, "2024-Q1", "2025-Q1")
evol_analysis

,Reviews end of 2024-Q1,Reviews end of 2025-Q1,Reviews Gained,% Change Reviews,Progression > 0%,check_reviews
ASIN,,,,,,
B00001P4ZH,NaN,7855.0,7855.0,inf,True,Augmentation
B00005K49U,0.0,3222.0,3222.0,inf,True,Augmentation
B00005K49W,0.0,3222.0,3222.0,inf,True,Augmentation
B00005K49X,0.0,3222.0,3222.0,inf,True,Augmentation
B00005K4A4,NaN,0.0,0.0,NaN,False,No reviews
...,...,...,...,...,...,...
B0DW9DSFBR,NaN,0.0,0.0,NaN,False,No reviews
B0DW9F5M5T,NaN,0.0,0.0,NaN,False,No reviews
B0DW9G3LQ3,NaN,0.0,0.0,NaN,False,No reviews


##### Stars

In [1009]:
evol = show_evol(stars_evolution, product)
evol

extract_date,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01
ASIN,,,,,,,,,,,,,,,
B08BNYFVFS,NaN,NaN,NaN,4.3,4.3,4.3,4.3,4.3,NaN,4.2,4.2,4.2,4.2,4.2,4.2


In [1010]:
evol_analysis = analyse_progression_stars(stars_evolution, "2024-Q2", "2025-Q1")
evol_analysis

,Stars end of 2024-Q2,Stars end of 2025-Q1,Delta Stars,Tendance,Delta significatif,check_stars
ASIN,,,,,,
B00001P4ZH,4.5,4.5,0.0,Stable,False,Stable
B00005K49U,4.6,4.6,0.0,Stable,False,Stable
B00005K49W,4.6,4.6,0.0,Stable,False,Stable
B00005K49X,4.6,4.6,0.0,Stable,False,Stable
B00005K4A4,NaN,NaN,NaN,Inconnu,False,Inconnu
...,...,...,...,...,...,...
B0DW9DSFBR,NaN,NaN,NaN,Inconnu,False,Inconnu
B0DW9F5M5T,NaN,NaN,NaN,Inconnu,False,Inconnu
B0DW9G3LQ3,NaN,NaN,NaN,Inconnu,False,Inconnu


##### Retours

In [1011]:
evol = show_evol(retours_evolution, product)
evol

extract_date,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01
ASIN,,,,,,,,,,,,,,,
B08BNYFVFS,1.0,9.0,3.0,3.0,1.0,1.0,1.0,1.0,0.0,2.0,3.0,0.0,0.0,1.0,0.0


In [1012]:
evol_analysis = analyse_progression_retours(evol, "2024-Q2", "2025-Q1")
evol_analysis

,Retours 2024-Q2 (moy),Retours 2025-Q1 (moy),Delta,% Changement,Tendance,Hausse problématique,check_returns
ASIN,,,,,,,
B08BNYFVFS,1.666667,0.333333,-4.0,-80.0,Baisse,False,Baisse


##### CVR

In [1013]:
cvr_evolution[cvr_evolution['2025-02-01'] >0 ]

extract_date,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01
ASIN,,,,,,,,,,,,,,,
B00005KHSV,NaN,NaN,NaN,14.29,0.00,33.33,0.00,0.00,NaN,0.00,0.00,0.00,NaN,12.50,NaN
B00005KHTD,NaN,NaN,NaN,NaN,NaN,0.00,11.11,0.00,NaN,4.55,0.00,3.85,NaN,5.00,NaN
B00005QF9N,NaN,NaN,NaN,9.09,7.46,6.70,6.08,6.09,NaN,4.16,5.83,4.48,5.05,6.78,4.05
B00005QFAY,NaN,NaN,NaN,10.34,21.13,16.96,23.38,17.26,NaN,11.28,13.51,12.88,16.63,9.94,18.59
B00005QFAZ,NaN,NaN,NaN,NaN,NaN,14.63,32.35,30.56,NaN,15.56,11.48,21.43,12.99,28.57,8.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B0DGL73H6Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.50,NaN
B0DGL96V5R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,NaN
B0DGLN4233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,3.70,8.33,5.71,10.53


In [1014]:
evol = show_evol(cvr_evolution, "B00005QFAY")
evol

extract_date,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01
ASIN,,,,,,,,,,,,,,,
B00005QFAY,NaN,NaN,NaN,10.34,21.13,16.96,23.38,17.26,NaN,11.28,13.51,12.88,16.63,9.94,18.59


In [1015]:
evol_analysis = analyse_progression_cvr(cvr_evolution)
evol_analysis

,First CVR,Last CVR,Delta CVR,Tendance,CVR Moyen,CVR Écart-type,Chute significative,CVR globalement bas,CVR instable
ASIN,,,,,,,,,
B00001P4ZH,0.00,0.00,0.00,Stable,0.000000,0.000000,False,True,False
B00005K49U,5.26,3.57,-1.69,Baisse,8.158571,6.834885,False,False,False
B00005K49W,0.00,0.00,0.00,Stable,0.000000,0.000000,False,True,False
B00005K49X,0.00,0.00,0.00,Stable,0.000000,0.000000,False,True,False
B00005K4A4,NaN,NaN,NaN,Stable,NaN,NaN,False,False,False
...,...,...,...,...,...,...,...,...,...
B0DW9DSFBR,NaN,NaN,NaN,Stable,NaN,NaN,False,False,False
B0DW9F5M5T,NaN,NaN,NaN,Stable,NaN,NaN,False,False,False
B0DW9G3LQ3,NaN,NaN,NaN,Stable,NaN,NaN,False,False,False


In [1016]:
evol_analysis.head(50)

,First CVR,Last CVR,Delta CVR,Tendance,CVR Moyen,CVR Écart-type,Chute significative,CVR globalement bas,CVR instable
ASIN,,,,,,,,,
B00001P4ZH,0.00,0.00,0.00,Stable,0.000000,0.000000,False,True,False
B00005K49U,5.26,3.57,-1.69,Baisse,8.158571,6.834885,False,False,False
B00005K49W,0.00,0.00,0.00,Stable,0.000000,0.000000,False,True,False
B00005K49X,0.00,0.00,0.00,Stable,0.000000,0.000000,False,True,False
B00005K4A4,NaN,NaN,NaN,Stable,NaN,NaN,False,False,False
B00005K4A5,14.29,2.63,-11.66,Baisse,5.034545,5.546255,True,False,False
B00005K4AG,NaN,NaN,NaN,Stable,NaN,NaN,False,False,False
B00005K4AH,NaN,NaN,NaN,Stable,NaN,NaN,False,False,False
B00005KHSU,NaN,NaN,NaN,Stable,NaN,NaN,False,False,False


##### TEST

In [1017]:
product = "B08BNYFVFS"
reviews_evolution = reviews_evolution[reviews_evolution.index == product]
progress = reviews_evolution.iloc[:, -1] - reviews_evolution.iloc[:, 0]
growth_trend = reviews_evolution.diff(axis=1) >= 0
nb_mois_positifs = growth_trend.sum(axis=1)
growth_trend
# diagnostic_reviews = (progress > 0) & (nb_mois_positifs >= 3)
# diagnostic_reviews

extract_date,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01
ASIN,,,,,,,,,,,,,,,
B08BNYFVFS,False,True,True,True,True,True,True,True,False,True,True,True,True,True,True
